In [1]:
import datetime
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import iqr
import math
from scipy import integrate
from sklearn.metrics import mean_squared_error
from scipy.integrate import romberg
from scipy.optimize import minimize
import os
import pandas as pd

def mean_absolute_error(y_true, y_pred):
    """
    计算平均绝对误差（MAE）
    
    参数：
    y_true: 实际观测值的数组或列表
    y_pred: 对应的预测值的数组或列表
    
    返回：
    mae: 平均绝对误差
    """
    n = len(y_true)
    error_sum = 0.0
    
    for i in range(n):
        error_sum += abs(y_pred[i] - y_true[i])
    
    mae = error_sum / n
    return mae
def mean_squared_error(y_true, y_pred):
    """
    计算均方误差（MSE）
    
    参数：
    y_true: 实际观测值的数组或列表
    y_pred: 对应的预测值的数组或列表
    
    返回：
    mse: 均方误差
    """
    n = len(y_true)
    squared_error_sum = 0.0
    
    for i in range(n):
        squared_error_sum += (y_pred[i] - y_true[i])**2
    
    mse = ( squared_error_sum ** 0.5) / n
    return mse
###Softmax函数转化
def softmax(x):
    e_x = np.exp(x)
    #e_x = np.exp(x - np.max(x))  # 减去最大值防止溢出
    return e_x / e_x.sum(axis=0)
# 定义被积函数
def estimated_function(x, X_train, Y_train, h_list, w_list, kernel_func, estimator):
    y_pred = estimator(X_train, Y_train, x, h_list, w_list, kernel_func)
    return y_pred ** 2

###复合核函数
def composite_kernel(t, h_list, w_list):
    # 各核函数的线性组合
    return (
        ##高斯核函数
        w_list[0] * (1 / h_list[0]) * (1 / np.sqrt(2 * np.pi)) * np.exp(-0.5 * (t / h_list[0]) ** 2) +
        ##biweight核函数
        w_list[1] * (1 - abs(t / h_list[1]) if abs(t / h_list[1]) <= 1 else 0) +
        ##triangular核函数
        w_list[2] * ((15 / 16) * (1 - (t / h_list[2]) ** 2) ** 2 if abs(t / h_list[2]) <= 1 else 0) +
        ##epchnikov核函数
        w_list[3] * (0.75 * (1 - (t / h_list[3]) ** 2 / 5) / np.sqrt(5) if abs(t / h_list[3]) <= np.sqrt(5) else 0) +
        ###qu核函数
        w_list[4] * (0.75 * (1 - (t / h_list[4]) ** 2) if abs(t / h_list[4]) <= 1 else 0)
    )

In [155]:
angular = [1, 2, 3, 7, 13, 23, 46, 97, 172, 263, 333, 396, 492, 536, 587, 653, 677, 711, 730, 761, 873, 939, 1009, 1107, 1196, 1253, 1296, 1331, 1411, 1616, 1721, 1817, 1925, 2020, 2096, 2176, 2245, 2336, 2423, 2517, 2587, 2655, 2719, 2821, 2885, 2933, 2990, 3039, 3095, 3137, 3191, 3252, 3295, 3344, 3421, 3504, 3584, 3646, 3707, 3761, 3827, 3915, 3972, 4031, 4157, 4222, 4280, 4333, 4375, 4412, 4448, 4482, 4495, 4504, 4510, 4515, 4521, 4528, 4529, 4540, 4547, 4554, 4561, 4568, 4573, 4578, 4584, 4589, 4593, 4597, 4619, 4641, 4648, 4662, 4667, 4675, 4679, 4687, 4700, 4703, 4713, 4714]
backbone = [12, 20, 21, 27, 30, 32, 34, 39, 45, 47, 47, 48, 52, 56, 64, 69, 72, 76, 79, 84, 86, 86, 89, 91, 92, 110, 123, 127, 138, 139, 144, 147, 148, 148, 148, 150, 153, 153, 153, 159, 159, 161, 161, 162, 163, 163, 164, 166, 166, 166, 168, 170, 171, 172, 175, 176, 177, 178, 179, 180, 180, 181, 181, 182, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 184, 184, 185, 185, 185, 185, 185, 186, 186, 186, 186, 186, 186, 186, 186, 187, 187, 187, 187, 187, 188, 188, 188, 188, 188, 188, 188, 189, 189, 189, 190]
brew = [7, 10, 18, 28, 37, 42, 44, 46, 48, 48, 50, 53, 54, 54, 56, 58, 59, 61, 62, 63, 64, 65, 66, 67, 69, 71, 72, 77, 80, 80, 80, 82, 85, 88, 88, 90, 92, 93, 93, 94, 96, 98, 98, 98, 101, 102, 102, 105, 106, 111, 114, 114, 114, 114, 118, 121, 122, 171, 204, 251, 271, 292, 316, 337, 364, 388, 394, 409, 428, 439, 456, 476, 493, 513, 529, 560, 594, 617, 639, 655, 673, 733, 757, 768]
ccxt = [6, 7, 8, 8, 8, 10, 10, 10, 10, 10, 11, 13, 14, 17, 26, 31, 33, 34, 43, 63, 74, 80, 86, 94, 102, 111, 116, 126, 142, 161, 173, 179, 183, 191, 195, 207, 222, 228, 242, 255, 261, 281, 304, 316, 332, 346, 351, 376, 394, 416, 443, 462, 481, 503, 534, 570, 600, 626, 644, 670, 689, 730, 792, 811, 846, 853]
chartjs = [1, 3, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 11, 20, 28, 34, 39, 45, 54, 61, 66, 69, 70, 84, 99, 108, 113, 117, 140, 163, 189, 216, 245, 305, 358, 401, 435, 474, 504, 524, 554, 569, 588, 616, 637, 658, 683, 713, 743, 775, 790, 804, 818, 827, 846, 861, 876, 887, 897, 907, 913, 927, 930, 936, 942, 953, 960, 969, 986, 999, 1010, 1022, 1046, 1063, 1075, 1098, 1113, 1130, 1160, 1176, 1196, 1210, 1242, 1277, 1300, 1313, 1334, 1361, 1401, 1429, 1449, 1484, 1529, 1566, 1603, 1630, 1673, 1701, 1727, 1744, 1771, 1787, 1800, 1813, 1826, 1841, 1857, 1869, 1886, 1909, 1928, 1949, 1986, 2010, 2036, 2052, 2069]
compose = [2, 2, 4, 4, 5, 6, 9, 10, 11, 18, 22, 28, 36, 47, 54, 64, 78, 95, 115, 134, 159, 180, 184, 202, 239, 248, 269, 283, 289, 299, 307, 314, 322, 333, 335, 350, 361, 370, 380, 388, 393, 399, 404, 411, 416, 419, 425, 435, 442, 451, 459, 466, 473, 474, 474, 474, 479, 488, 491, 498, 503, 508, 515, 520, 527, 530, 534, 535, 551, 573, 586, 612, 631, 662, 702, 737, 766, 792, 822, 861, 885, 911, 928, 963, 998, 1041, 1068, 1084, 1107, 1130, 1162, 1218, 1240, 1243, 1243, 1243, 1243, 1244, 1244, 1246, 1251, 1254, 1263, 1268, 1296, 1321, 1346, 1372, 1412, 1437, 1446]
curl = [2, 2, 2, 3, 3, 3, 3, 6, 6, 7, 8, 10, 14, 16, 21, 23, 23, 29, 33, 38, 41, 41, 43, 47, 49, 52, 56, 57, 61, 65, 73, 81, 89, 95, 98, 109, 115, 117, 124, 135, 139, 145, 158, 163, 170, 178, 185, 188, 199, 209, 224, 232, 238, 247, 259, 271, 282, 292, 303, 321, 332, 344, 357, 366, 375, 382, 395, 409, 421, 428, 436, 449, 470, 479, 496, 515, 523, 536, 543, 552, 560, 576, 582, 595, 602, 610, 614, 621, 636, 651, 662, 674, 686, 705, 719, 727]
electron = [1, 1, 3, 3, 6, 12, 14, 16, 18, 20, 25, 30, 37, 43, 50, 61, 67, 76, 91, 102, 114, 123, 139, 144, 161, 173, 187, 199, 216, 229, 237, 261, 325, 356, 385, 438, 507, 564, 636, 720, 800, 872, 930, 988, 1059, 1142, 1209, 1312, 1413, 1505, 1607, 1698, 1793, 1849, 1907, 1975, 2039, 2109, 2179, 2261, 2322, 2384, 2460, 2515, 2609, 2673, 2716, 2764, 2835, 2905, 2956, 3009, 3069, 3125, 3190, 3258, 3334, 3398, 3456, 3516, 3580, 3710, 3844, 3959, 4070, 4173, 4272, 4398, 4512, 4639, 4742, 4845, 4974, 5083, 5209, 5297, 5395, 5496, 5590, 5673, 5738, 5817, 5905, 5972, 6070, 6150]
framework = [6, 8, 8, 8, 9, 10, 11, 11, 12, 12, 12, 12, 13, 13, 13, 14, 15, 17, 28, 35, 56, 65, 70, 79, 82, 92, 99, 102, 111, 116, 123, 127, 132, 137, 164, 167, 172, 178, 181, 187, 195, 195, 197, 198, 198, 199, 202, 203, 204, 208, 213, 213, 214, 215, 218, 222, 223, 226, 229, 230, 234, 238, 239, 242, 242, 245, 249, 251, 261, 283, 293, 302, 314, 328, 343, 352, 365, 373, 390, 406, 408, 414, 420, 434, 448, 480, 496, 502, 508, 515, 523, 534, 541, 548, 555, 565, 571, 581, 585, 592, 603, 606, 613, 618, 625, 629, 631, 640, 656, 666, 673, 680, 686, 691, 698, 702, 703, 704, 705, 708, 711, 712, 720]
netty = [14, 19, 24, 25, 38, 58, 71, 90, 119, 139, 159, 183, 209, 236, 270, 308, 359, 386, 412, 482, 512, 533, 548, 560, 576, 595, 619, 632, 652, 661, 681, 708, 741, 769, 804, 815, 834, 855, 865, 883, 909, 928, 945, 963, 985, 1010, 1023, 1037, 1053, 1078, 1094, 1113, 1134, 1152, 1167, 1186, 1200, 1215, 1224, 1244, 1258, 1278, 1291, 1307, 1320, 1334, 1344, 1362, 1376, 1394, 1402, 1412, 1419, 1434, 1442, 1450, 1456, 1457, 1461, 1465, 1471, 1473, 1478, 1485, 1490, 1495, 1499, 1501, 1505, 1509, 1512, 1516, 1517, 1517, 1519, 1519, 1521, 1523, 1524, 1524, 1525, 1527, 1528, 1528, 1528, 1528, 1528, 1528, 1528, 1529, 1529, 1530, 1530, 1530, 1530, 1530, 1530, 1530, 1531, 1531, 1531, 1531, 1531, 1531, 1531, 1531, 1531, 1531, 1532, 1532, 1532, 1532, 1532, 1532, 1535, 1536]
nextJS = [22, 45, 81, 106, 115, 126, 144, 161, 168, 172, 176, 181, 187, 192, 197, 214, 224, 229, 245, 258, 276, 283, 304, 322, 332, 342, 351, 375, 389, 410, 423, 430, 488, 503, 522, 556, 584, 603, 638, 659, 699, 749, 811, 931, 1046, 1138, 1201, 1291, 1519, 1664, 1857, 2030, 2241, 2434, 2629, 2889, 3063, 3265, 3428, 3694, 4011, 4189, 4371, 4560, 4749, 4945, 5112, 5252, 5432, 5620, 5794, 5982, 6320, 6539, 6723, 6958, 7188]
pandas = [2, 2, 2, 2, 2, 2, 2, 3, 4, 8, 13, 30, 53, 92, 134, 181, 221, 254, 317, 380, 448, 526, 583, 639, 689, 764, 829, 892, 938, 989, 1054, 1103, 1167, 1222, 1275, 1331, 1385, 1433, 1479, 1556, 1615, 1660, 1703, 1757, 1829, 1901, 1948, 1998, 2050, 2103, 2150, 2185, 2236, 2278, 2326, 2367, 2414, 2472, 2501, 2536, 2586, 2638, 2682, 2728, 2786, 2838, 2905, 2976, 3020, 3090, 3154, 3197, 3241, 3278, 3322, 3368, 3436, 3494, 3546, 3604, 3665, 3732, 3774, 3814, 3864, 3960, 4027, 4105, 4160, 4218, 4281, 4354, 4430, 4505, 4580, 4654, 4725, 4835, 4917, 4987, 5066, 5142, 5189, 5242, 5309, 5374, 5431, 5504, 5575, 5651, 5703, 5799, 5897, 5999, 6116, 6232, 6344, 6440, 6555, 6655, 6754, 6846, 6953, 7074, 7186, 7293, 7385, 7459, 7551, 7648, 7734, 7824, 7907, 7971, 8040, 8133, 8230, 8315, 8390, 8451, 8510, 8576, 8629, 8731, 8825, 8903, 9010, 9125, 9233, 9344]
pytorch = [1, 3, 6, 10, 13, 19, 24, 29, 31, 34, 35, 39, 42, 43, 46, 49, 52, 54, 62, 65, 65, 66, 73, 76, 87, 91, 103, 115, 124, 139, 156, 169, 189, 208, 228, 251, 267, 279, 293, 315, 324, 334, 355, 373, 386, 459, 477, 493, 506, 519, 534, 560, 570, 586, 610, 621, 633, 647, 661, 684, 700, 712, 732, 755, 789, 811, 825, 838, 859, 886, 909, 960, 1027, 1098, 1153, 1196, 1234, 1266]
rails = [1, 1, 1, 1, 1, 2, 2, 2, 3, 4, 4, 7, 8, 8, 8, 8, 8, 9, 10, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 14, 17, 17, 19, 19, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 23, 29, 32, 36, 39, 44, 47, 50, 50, 51, 52, 56, 62, 67, 76, 83, 84, 91, 91, 91, 92, 93, 93, 93, 94, 94, 94, 96, 98, 98, 98, 100, 101, 102, 102, 102, 106, 106, 107, 108, 108, 108, 108, 109, 110, 110, 112, 115, 121, 123, 126, 127, 127, 128, 128, 128, 128, 129, 129, 130, 130, 132, 132, 132, 136, 137, 140, 145, 146, 147, 148, 148, 150, 151, 155, 157, 157, 162, 164, 166, 166, 167, 171, 173, 180, 188, 191, 194]
react = [6, 9, 10, 12, 23, 26, 31, 33, 37, 39, 43, 46, 53, 55, 64, 68, 71, 73, 74, 76, 83, 89, 91, 92, 99, 101, 103, 104, 105, 106, 108, 110, 121, 127, 133, 140, 151, 154, 160, 160, 161, 168, 173, 177, 180, 188, 193, 200, 212, 217, 228, 249, 277, 297, 306, 315, 324, 336, 342, 346, 350, 359, 364, 371, 388, 393, 397, 410, 418, 423, 425, 428, 429, 479, 501, 519, 533, 574, 601, 612, 619, 632, 646, 657, 679, 702, 716, 737, 755, 767, 776, 789, 805, 839, 877, 899, 918, 932, 948, 963, 982, 997, 1009, 1024, 1037, 1056, 1072, 1083, 1093, 1100, 1108, 1115, 1119, 1150, 1160, 1167, 1174]
redis = [21, 27, 33, 37, 46, 49, 53, 56, 60, 61, 62, 65, 67, 70, 71, 75, 75, 77, 77, 79, 79, 80, 80, 80, 80, 80, 81, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 83, 84, 86, 87, 88, 89, 89, 91, 93, 93, 94, 94, 95, 97, 104, 105, 105, 106, 107, 110, 112, 112, 113, 113, 119, 122, 125, 126, 127, 127, 128, 129, 129, 132, 134, 135, 136, 136, 136, 136, 136, 136, 136, 136, 136, 137, 137, 137, 137, 137, 138, 138, 138, 138, 138, 138, 138, 139, 141, 141, 142, 143, 143, 143, 143, 144, 146, 146, 146, 146, 146, 146, 146, 146, 148, 148, 148, 148, 148, 148, 148, 148, 150]
sails = [1, 2, 2, 4, 4, 8, 15, 19, 37, 46, 57, 64, 80, 85, 92, 97, 103, 105, 108, 124, 139, 147, 154, 157, 157, 161, 165, 167, 173, 174, 178, 180, 188, 203, 205, 205, 207, 208, 210, 212, 215, 215, 216, 217, 219, 222, 226, 229, 233, 235, 235, 238, 240, 240, 246, 246, 247, 249, 249, 251, 254, 256, 256, 256, 257, 257, 258, 260, 260, 261, 264, 264, 264, 264, 265, 265, 265, 265, 265, 266, 266, 266, 266, 266, 266, 267]
styled_components = [17, 38, 45, 59, 77, 84, 94, 112, 122, 126, 134, 138, 152, 155, 159, 166, 172, 179, 184, 186, 188, 189, 190, 194, 200, 205, 205, 209, 210, 211, 211, 214, 219, 220, 220, 223, 227, 227, 229, 235, 238, 240, 241, 242, 243, 244, 244, 246, 246, 247, 248, 248, 248, 248, 248, 249, 250, 251, 251, 251, 251, 251, 251, 251, 252, 252, 252, 252, 252, 252, 252, 252, 252, 252, 253, 254]
vue = [3, 3, 4, 6, 7, 12, 14, 16, 16, 17, 17, 17, 20, 22, 22, 23, 24, 26, 27, 29, 37, 40, 46, 52, 56, 58, 60, 64, 70, 76, 83, 97, 114, 127, 138, 158, 169, 181, 192, 203, 214, 214, 221, 228, 236, 243, 252, 263, 268, 273, 280, 282, 288, 297, 299, 302, 305, 307, 309, 314, 316, 322, 329, 334, 345, 348, 354, 356, 361, 364, 369, 373, 376, 378, 380, 383, 385, 388, 389, 390, 391, 393, 394, 396, 397, 398, 398, 399, 400, 400, 400, 400, 400, 402, 402, 403, 403, 403, 403, 403, 403, 403, 403, 404, 407, 407, 407, 407, 407, 408]
wox = [2, 10, 11, 12, 12, 13, 16, 17, 19, 19, 20, 22, 26, 27, 29, 31, 33, 34, 37, 37, 37, 50, 58, 66, 69, 81, 90, 120, 130, 137, 144, 148, 154, 157, 157, 161, 166, 176, 181, 185, 190, 191, 192, 193, 193, 195, 199, 208, 209, 209, 209, 209, 209, 209, 211, 211, 211, 211, 212, 212, 213, 215, 215, 216, 216, 216, 216, 216, 217, 217, 217, 217, 217, 217, 224, 258, 269, 275, 287, 295, 308, 313, 314, 320, 328, 329, 330, 332, 334, 340, 344, 348, 350, 353, 353, 354, 354, 354, 355, 356, 359, 362, 362, 362, 362, 365, 367, 368, 368, 371]
xstate = [2, 5, 7, 11, 16, 18, 18, 19, 23, 24, 28, 36, 43, 44, 46, 49, 52, 55, 56, 66, 95, 115, 133, 146, 156, 170, 182, 197, 198, 202, 203, 205, 207, 209, 214, 216, 216, 217, 221, 222, 224, 227, 230, 232, 232, 234, 237, 240, 264, 279, 288, 301, 307, 318, 337, 344, 350, 352, 356, 365, 378, 383]
yarn = [1, 7, 17, 22, 35, 172, 233, 259, 277, 296, 321, 344, 385, 420, 445, 458, 495, 510, 536, 549, 571, 584, 606, 627, 642, 658, 670, 684, 688, 694, 703, 712, 718, 723, 726, 726, 727, 728, 729, 729, 729, 730, 740, 748, 751, 756, 769, 770, 770, 770, 770, 772, 772, 776, 776, 776, 776, 776, 776, 776, 776, 776, 776, 776, 779, 779, 779, 779, 780, 780, 784, 787, 790, 790, 791, 792, 792, 792, 792, 792, 792, 793]
zig = [12, 14, 15, 24, 25, 25, 26, 27, 34, 36, 37, 37, 39, 46, 52, 68, 79, 88, 89, 96, 106, 125, 141, 151, 176, 194, 222, 243, 256, 304, 338, 371, 411, 426, 442, 451, 464, 484, 506, 535, 580, 627, 656, 704, 765, 810, 862, 910, 968, 1031, 1083, 1145, 1200, 1249, 1276, 1313, 1349, 1439, 1510, 1570, 1624, 1658, 1722, 1767, 1822, 1877, 1917, 1950, 1983, 2033, 2080, 2158, 2231, 2298, 2398, 2463, 2515, 2590, 2733, 2919, 3069, 3233, 3392, 3546, 3659, 3746, 3861, 3903]
DST1=[1,2,4,5,13,22,28,35,39,42,42,46,47,47,49,51,54]
DST2=[2,13,15,18,22,23,24,26,30,30,34,35,38,38]
DST3=[13,18,26,34,40,48,61,75,84,89,95,100,104,110,112,114,117,118,120]
DST4=[6,9,13,20,28,40,48,54,57,59,60,61]
#DST5=[0,1,2,3,3,4,5,6,6,7,7,8,9,9]
DST6=[3,19,28,31,33,33,35,36,37,39,48,51,53,54,56,63,66,66,66,66]
DST7=[0,1,1,6,6,6,8,9,10,10,16,20,20,20,20,26,29,32,36,38,44,46,46,52,53,55,55,56,57,58,58,58,58]
DST8=[0,0,0,0,0,2,5,5,5,7,7,10,29,37,42,42,42,44,45,47,49,49,49,49,50,50,52,52,52]

DST = dict()

#DST["DST1"] = DST1
#DST["DST2"] = DST2
#DST["DST3"] = DST3
#DST["DST4"] = DST4
#DST["DST5"] = DST5
#DST["DST6"] = DST6
#DST["DST7"] = DST7
#DST["DST8"] = DST8
#DST["angular"] = angular
#DST["backbone"] = backbone
#DST["brew"] = brew
#DST["ccxt"] = ccxt
#DST["chartjs"] = chartjs 
#DST["compose"] = compose
#DST["curl"] = curl
#DST["electron"] = electron
#DST["framework"] = framework
#DST["netty"] = netty
#DST["nextJS"] = nextJS
#DST["pandas"] = pandas
#DST["pytorch"] = pytorch
#DST["rails"] = rails
#DST["react"] = react
#DST["redis"] = redis
DST["sails"] = sails 
#DST["styled_components"] = styled_components
#DST["vue"] = vue
#DST["wox"] = wox
#DST["xstate"] = xstate
#DST["yarn"] = yarn
#DST["zig"] = zig

In [35]:
##NW估计器
def NW_estimator(X_train, y_train, x_test, h_list, w_list, kernel_func):
    n = len(X_train)
    numerator = 0.0
    denominator = 0.0
    
    for i in range(n):
        kernel = kernel_func(x_test - X_train[i], h_list, w_list)
        numerator += kernel * y_train[i]
        denominator += kernel
#        print("numerator",numerator)
#        print("denominator",denominator)
    
    if denominator != 0:
        y_pred = numerator / denominator
#        print(y_pred)
    else:
        y_pred = 0.0
        
    return y_pred
##ll估计器
def LL_estimator(X_train, y_train, x_test, h_list, w_list, kernel_func):
    n = len(X_train)
    numerator = 0.0
    denominator = 0.0
    sn1 = 0.0
    sn2 = 0.0
    
    # 预先计算用于局部线性估计的sn1和sn2
    for i in range(n):
        kernel_value = kernel_func(x_test - X_train[i], h_list, w_list)
        diff = x_test - X_train[i]
        sn1 += kernel_value * diff
        sn2 += kernel_value * (diff ** 2)
    
    # 计算权重w
    for i in range(n):
        kernel_value = kernel_func(x_test - X_train[i], h_list, w_list)
        diff = x_test - X_train[i]
        w = kernel_value * (sn2 - diff * sn1)
        numerator += w * y_train[i]
        denominator += w
    
    # 避免分母为零的情况
    y_pred = numerator / denominator if denominator != 0 else 0.0
    return y_pred

In [36]:
# 定义LSCV分数计算函数
def lscv_score(h_list, w_list, X, Y, kernel_func, estimator):
    # 确定积分区间和精度
    a, b = 0, 1
    integral_result = romberg(
        estimated_function, a, b,
        args=(X, Y, h_list, w_list, kernel_func, estimator),
        tol=1e-8
    )

    # 留一交叉验证计算
    loo_y_pred = []
    for i in range(len(X)):
        X_train = np.delete(X, i)
        Y_train = np.delete(Y, i)
        X_test = X[i]
        y_pred = estimator(X_train, Y_train, X_test, h_list, w_list, kernel_func)
        loo_y_pred.append(y_pred)

    # 计算 LSCV 分数
    score = integral_result - 2 * np.mean(loo_y_pred)
    return score

In [107]:
def find_optimal_parameters_cobyla(X, Y, kernel_func, initial_h_list, initial_w_list, bound_1, bound_ep, estimators):
    """
    使用 COBYLA 优化来找到最小化 LSCV 分数的最优 h_list 和 w_list，加入惩罚项处理权重之和为1的约束
    
    参数：
    X: 训练数据的自变量数组
    Y: 训练数据的因变量数组
    kernel_func: 核函数
    initial_h_list: 初始带宽列表
    initial_w_list: 初始权重列表
    bound_1: 特定带宽的下界
    bound_ep: ep核带宽的下界
    estimators: 估计器函数
    
    返回：
    optimal_h_list: 最优带宽列表
    optimal_w_list: 最优权重列表
    optimal_score: 最小的 LSCV 分数
    """
    # 定义带宽和权重边界范围
    h_bounds = [(1e-5, 1.0), (bound_1, 1.0), (bound_1, 1.0), (bound_ep, 1.0), (bound_1, 1.0)]
    w_bounds = [(0.0, 1.0) for _ in range(len(initial_w_list))]
    bounds = h_bounds + w_bounds
    
    # 将带宽和权重初始值组合成一个数组
    initial_guess = np.concatenate([initial_h_list, initial_w_list])
    
    # 定义惩罚项权重，使权重之和接近1
    penalty_weight = 1000

    # 目标函数，带惩罚项
    def objective(params):
        h_list = params[:len(initial_h_list)]
        w_list = params[len(initial_h_list):]
        # 权重和约束的惩罚项
        penalty = penalty_weight * abs(np.sum(w_list) - 1)
        return lscv_score(h_list, w_list, X, Y, kernel_func, estimators) + penalty

    # 使用 COBYLA 优化
    result = minimize(
        objective,
        initial_guess,
        method='COBYLA',
        bounds=bounds
    )
    
    if result.success:
        # 提取最优参数
        optimal_h_list = result.x[:len(initial_h_list)]
        optimal_w_list = result.x[len(initial_h_list):]
        optimal_score = result.fun
        return optimal_h_list, optimal_w_list, optimal_score
    else:
        raise ValueError("Optimization failed: " + result.message)


In [108]:
def find_optimal_parameters_pso(X, Y, kernel_func, initial_h_list, initial_w_list, bound_1, bound_ep, estimators):
    """
    使用粒子群优化来找到最小化LSCV分数的最优h_list和w_list
    
    参数：
    X: 训练数据的自变量数组
    Y: 训练数据的因变量数组
    kernel_func: 核函数
    initial_h_list: 初始带宽列表
    initial_w_list: 初始权重列表
    bound_1: 特定带宽的下界
    bound_ep: ep核带宽的下界
    estimators: 估计器函数
    
    返回：
    optimal_h_list: 最优带宽列表
    optimal_w_list: 最优权重列表
    optimal_score: 最小的LSCV分数
    """
    # 定义带宽的边界范围
    h_bounds = [
        (1e-5, 1.0),          # h1 的边界
        (bound_1, 1.0),    # h2 的边界
        (bound_1, 1.0),    # h3 的边界
        (bound_ep, 1.0),   # h4 的边界
        (bound_1, 1.0)     # h5 的边界
    ]
    
    # 定义权重的边界范围
    w_bounds = [(0.0, 1.0) for _ in range(len(initial_w_list))]

    # 将带宽和权重初始值组合成一个数组
    initial_guess = np.concatenate([initial_h_list, initial_w_list])
    
    # 定义约束条件，确保权重之和为 1
    def weight_constraint(w_list):
        return np.sum(w_list) - 1
    
    constraints = {'type': 'eq', 'fun': lambda params: weight_constraint(params[len(initial_h_list):])}
    
    # 定义目标函数
    def objective(params):
        h_list = params[:len(initial_h_list)]
        w_list = params[len(initial_h_list):]
        return lscv_score(h_list, w_list, X, Y, kernel_func, estimators)
    
    # 使用 minimize 函数进行优化
    result = minimize(
        objective,
        initial_guess,
        bounds=h_bounds + w_bounds,
        constraints=constraints,
        method='SLSQP'
    )
    
    if result.success:
        # 分离出最优的 h_list 和 w_list
        optimal_h_list = result.x[:len(initial_h_list)]
        optimal_w_list = result.x[len(initial_h_list):]
        optimal_score = result.fun
        return optimal_h_list, optimal_w_list, optimal_score
    else:
        raise ValueError("Optimization failed: " + result.message)

In [156]:
estimators = {
    "LL_estimator": LL_estimator # 主函数

}
def file_exists(filename):
    return os.path.exists(filename)

# 定义一个函数，用于将部分结果追加保存到 Excel 文件中
def save_partial_results(results, filename='Multy_LL_MIN_composite_kernel_results.xlsx'):
    temp_df = pd.DataFrame(results)
    sheet_name = f"Results_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
    with pd.ExcelWriter(filename, mode='a' if file_exists(filename) else 'w', engine='openpyxl') as writer:
        temp_df.to_excel(writer, index=False, sheet_name=sheet_name)
    results.clear()  # 清空results列表，以便收集新的部分结果

def main(DST, estimators, kernel_func):
    results = []  # 用来收集所有结果的列表

    for estimator_name, estimator in estimators.items():
        print(estimator_name, "---------------------------------------------------------------------------------------")
        
        for dataset_name, DS in DST.items():
            print(dataset_name, "---------------------------------------------------------------------------------------")
            number = len(DS)
            predictionLimit = number  # 预测限制
            initial_h_list = [0.5,0.5,0.5,0.5,0.5]
            initial_w_list = [0.6,0.1,0.1,0.1,0.1]
            start1 = [math.ceil(number * 0.2), math.ceil(number * 0.5), math.ceil(number * 0.8)]
            #start1 = [math.ceil(number * 0.5), math.ceil(number * 0.8)]
            #start1 = [ math.ceil(number * 0.8)]

            for start in start1:
                print(start / number)
                X = [i / start for i in range(1, start + 1)]
                Y = DS[:start]
                ##边界
                bound_1 = 1 / start
                bound_ep = 1 / start * 2.236
                ##参数寻优
                ##输出最优参数和最优得分 记得权重参数是没变化的参数
                optimal_h_list,optimal_w_list, optimal_score =  find_optimal_parameters_pso(X, Y, kernel_func, initial_h_list, initial_w_list, bound_1, bound_ep, estimator)
                Y_train = Y.copy()
                for i in range(start, predictionLimit):
                    X_train = [j / (i + 1) for j in range(1, i + 1)]
                    old_x_test = i / (i + 1)
                    X_test = 1

                    # 使用单个 estimator 函数
                    res = estimator(X_train, Y_train, X_test,optimal_h_list,optimal_w_list, kernel_func) - estimator(X_train, Y_train, old_x_test,optimal_h_list,optimal_w_list, kernel_func)
                    Y_train.append(Y_train[-1] + res)

                actual_values = DS[start:predictionLimit]
                predicted_values = Y_train[start:predictionLimit]
                Pmae = mean_absolute_error(actual_values, predicted_values)
                Pmse = mean_squared_error(actual_values, predicted_values)

                # 收集结果
                results.append({
                    'Estimator': estimator_name,
                    'Kernel': kernel_func.__name__,
                    'Dataset': dataset_name,
                    'Start': start,
                    'optimal_h_list': optimal_h_list,
                    'optimal_w_list': optimal_w_list,
                    'optimal_score': optimal_score,
                    'PMAE': Pmae,
                    'PMSE': Pmse,
                    'Y_train': ','.join(map(str, Y_train))
                })

                # 每次完成一个 start 就保存一次部分结果
                save_partial_results(results)

# 假设DST是一个包含数据集的字典，estimators是一个包含估计器的字典
main(DST, estimators, composite_kernel)

LL_estimator ---------------------------------------------------------------------------------------
sails ---------------------------------------------------------------------------------------
0.20930232558139536
0.5


D:\python\lib\site-packages\scipy\integrate\_quadrature.py:849: AccuracyWarning: divmax (10) exceeded. Latest difference = 1.988732e-03
  warnings.warn(
D:\python\lib\site-packages\scipy\integrate\_quadrature.py:849: AccuracyWarning: divmax (10) exceeded. Latest difference = 1.988734e-03
  warnings.warn(
D:\python\lib\site-packages\scipy\integrate\_quadrature.py:849: AccuracyWarning: divmax (10) exceeded. Latest difference = 1.988731e-03
  warnings.warn(
D:\python\lib\site-packages\scipy\integrate\_quadrature.py:849: AccuracyWarning: divmax (10) exceeded. Latest difference = 1.988733e-03
  warnings.warn(
D:\python\lib\site-packages\scipy\integrate\_quadrature.py:849: AccuracyWarning: divmax (10) exceeded. Latest difference = 5.979070e-04
  warnings.warn(
D:\python\lib\site-packages\scipy\integrate\_quadrature.py:849: AccuracyWarning: divmax (10) exceeded. Latest difference = 5.979069e-04
  warnings.warn(
D:\python\lib\site-packages\scipy\integrate\_quadrature.py:849: AccuracyWarning: d

D:\python\lib\site-packages\scipy\integrate\_quadrature.py:849: AccuracyWarning: divmax (10) exceeded. Latest difference = 8.534324e-03
  warnings.warn(
D:\python\lib\site-packages\scipy\integrate\_quadrature.py:849: AccuracyWarning: divmax (10) exceeded. Latest difference = 5.309936e-03
  warnings.warn(
D:\python\lib\site-packages\scipy\integrate\_quadrature.py:849: AccuracyWarning: divmax (10) exceeded. Latest difference = 4.838039e-03
  warnings.warn(
D:\python\lib\site-packages\scipy\integrate\_quadrature.py:849: AccuracyWarning: divmax (10) exceeded. Latest difference = 4.838150e-03
  warnings.warn(
D:\python\lib\site-packages\scipy\integrate\_quadrature.py:849: AccuracyWarning: divmax (10) exceeded. Latest difference = 7.525615e-03
  warnings.warn(
D:\python\lib\site-packages\scipy\integrate\_quadrature.py:849: AccuracyWarning: divmax (10) exceeded. Latest difference = 5.893125e-03
  warnings.warn(
D:\python\lib\site-packages\scipy\integrate\_quadrature.py:849: AccuracyWarning: d

0.8023255813953488


D:\python\lib\site-packages\scipy\integrate\_quadrature.py:849: AccuracyWarning: divmax (10) exceeded. Latest difference = 3.612585e-01
  warnings.warn(
D:\python\lib\site-packages\scipy\integrate\_quadrature.py:849: AccuracyWarning: divmax (10) exceeded. Latest difference = 3.568749e-01
  warnings.warn(
D:\python\lib\site-packages\scipy\integrate\_quadrature.py:849: AccuracyWarning: divmax (10) exceeded. Latest difference = 7.586361e-04
  warnings.warn(
D:\python\lib\site-packages\scipy\integrate\_quadrature.py:849: AccuracyWarning: divmax (10) exceeded. Latest difference = 7.586360e-04
  warnings.warn(
D:\python\lib\site-packages\scipy\integrate\_quadrature.py:849: AccuracyWarning: divmax (10) exceeded. Latest difference = 7.586359e-04
  warnings.warn(
D:\python\lib\site-packages\scipy\integrate\_quadrature.py:849: AccuracyWarning: divmax (10) exceeded. Latest difference = 7.586482e-04
  warnings.warn(
D:\python\lib\site-packages\scipy\integrate\_quadrature.py:849: AccuracyWarning: d

D:\python\lib\site-packages\scipy\integrate\_quadrature.py:849: AccuracyWarning: divmax (10) exceeded. Latest difference = 6.797007e-04
  warnings.warn(
D:\python\lib\site-packages\scipy\integrate\_quadrature.py:849: AccuracyWarning: divmax (10) exceeded. Latest difference = 6.797045e-04
  warnings.warn(
D:\python\lib\site-packages\scipy\integrate\_quadrature.py:849: AccuracyWarning: divmax (10) exceeded. Latest difference = 6.796612e-04
  warnings.warn(
D:\python\lib\site-packages\scipy\integrate\_quadrature.py:849: AccuracyWarning: divmax (10) exceeded. Latest difference = 6.797005e-04
  warnings.warn(
D:\python\lib\site-packages\scipy\integrate\_quadrature.py:849: AccuracyWarning: divmax (10) exceeded. Latest difference = 1.544549e-03
  warnings.warn(
D:\python\lib\site-packages\scipy\integrate\_quadrature.py:849: AccuracyWarning: divmax (10) exceeded. Latest difference = 8.317214e-04
  warnings.warn(
D:\python\lib\site-packages\scipy\integrate\_quadrature.py:849: AccuracyWarning: d

In [123]:
DS = rails
number = len(DS)
predictionLimit = number #预测限制
#start1 = [math.ceil(number*0.2),math.ceil(number*0.5),math.ceil(number*0.8)]
start1 = [math.ceil(number*0.5)]
kernel_func = composite_kernel
estimators = NW_estimator
initial_h_list = [0.5,0.5,0.5,0.5,0.5]
initial_w_list = [0.2,0.2,0.2,0.2,0.2]
for start in start1:
    print(start / number)
    X = []
    Y = []
#       res = []
    for i in range(1, start +1 ):
        X.append( i  / start )
    for i in range(start):
        Y.append(DS[i])
    #想用部分核函数需要边界设置
    bound_1 = 1 / start
    bound_ep =  1 / start * 2.236 
    ###粒子群边界
    #h_bounds = [
    #(0, 1.0),  # h1 的边界
    #(bound_1, 1.0),  # h2 的边界
    #(bound_1, 1.0),  # h3 的边界
    #(bound_ep, 1.0),  # h4 的边界
    #(bound_1, 1.0)   # h5 的边界
    #]  
    #w_bounds = (0.0, 1.0)  # w的边界
    ##输出最优参数和最优得分 记得权重参数是没变化的参数
    optimal_h_list,optimal_w_list, optimal_score = find_optimal_parameters_pso(X, Y, kernel_func, initial_h_list, initial_w_list, bound_1, bound_ep, estimators)
    Y_train = Y
    i = start 
    print(i)
    while i < predictionLimit:
        X_train = []
        for j in range(1,i+1):
            X_train.append( j / (i +1)  )
        old_x_test =( i  ) / ( i + 1 )
        X_test = 1 
        res = estimators(X_train, Y_train, X_test,optimal_h_list,optimal_w_list, kernel_func) - estimators(X_train, Y_train, old_x_test,optimal_h_list,optimal_w_list, kernel_func)
        print("old_x_test")
        print(old_x_test)
        print("X_test")
        print(X_test)
        print("res")
        print(res)
        Y_train.append(Y_train[-1] + res)
        i +=1
    print(Y_train)
    print(start)
    print(number)
    actual_values = DS[start:number]
    predicted_values = Y_train[start:number]
    Pmae = mean_absolute_error(actual_values,predicted_values)
    Pmse = mean_squared_error(actual_values,predicted_values)
    print("pmAe",Pmae)
    print("Pmse",Pmse)

0.5


KeyboardInterrupt: 

In [8]:
print(optimal_h_list)

[1.         0.99945999 0.9990704  0.45215921 1.        ]


In [9]:
print(optimal_w_list)

[6.60412159e-15 3.09974034e-14 6.18411103e-15 1.83889629e-01
 8.16110371e-01]


In [10]:
print(sum(optimal_w_list))

0.9999999999999999
